## Function Calling

In [1]:
from langchain_core.tools import tool

@tool
def guess_tribe(name: str) -> str:
    """Guesses your tribe based on your name

    Args:
        name: your name
    """
    if "A" < name[0]  < "M":
        return "Zulu"
    
    elif "M" <= name[0] < "R":
        return "Yoruba"
    else:
        return "Isoko"

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
model_with_tools = model.bind_tools([guess_tribe])

response = model_with_tools.invoke("My friends are Chloe, Ryan, James and Quentin, could you tell me their heritage?")

In [3]:
from pprint import pprint
pprint(response.additional_kwargs)

{'refusal': None,
 'tool_calls': [{'function': {'arguments': '{"name": "Chloe"}',
                              'name': 'guess_tribe'},
                 'id': 'call_G4W45cKuTUPT6awzw1S9ezfF',
                 'type': 'function'},
                {'function': {'arguments': '{"name": "Ryan"}',
                              'name': 'guess_tribe'},
                 'id': 'call_gBP7ZKtqhEcz9yuNiL58fVDy',
                 'type': 'function'},
                {'function': {'arguments': '{"name": "James"}',
                              'name': 'guess_tribe'},
                 'id': 'call_4FnDOhwo9yDGXg2WXgHF0nYY',
                 'type': 'function'},
                {'function': {'arguments': '{"name": "Quentin"}',
                              'name': 'guess_tribe'},
                 'id': 'call_aCZMRwt1cusSh0EQRG1sN6WO',
                 'type': 'function'}]}


In [4]:
response.tool_calls

[{'name': 'guess_tribe',
  'args': {'name': 'Chloe'},
  'id': 'call_G4W45cKuTUPT6awzw1S9ezfF',
  'type': 'tool_call'},
 {'name': 'guess_tribe',
  'args': {'name': 'Ryan'},
  'id': 'call_gBP7ZKtqhEcz9yuNiL58fVDy',
  'type': 'tool_call'},
 {'name': 'guess_tribe',
  'args': {'name': 'James'},
  'id': 'call_4FnDOhwo9yDGXg2WXgHF0nYY',
  'type': 'tool_call'},
 {'name': 'guess_tribe',
  'args': {'name': 'Quentin'},
  'id': 'call_aCZMRwt1cusSh0EQRG1sN6WO',
  'type': 'tool_call'}]

In [7]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply a and b."""
    return a * b

llm_with_tools = model.bind_tools([multiply])

In [8]:
result_a = llm_with_tools.invoke("What is 2 multiplied by 3?")
pprint(result_a)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_XLI6Vla4MLH8aobHwzHoyG0n', 'function': {'arguments': '{"a":2,"b":3}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 54, 'total_tokens': 72, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-26e34d30-3134-4f82-afcc-f8012de6a541-0', tool_calls=[{'name': 'multiply', 'args': {'a': 2, 'b': 3}, 'id': 'call_XLI6Vla4MLH8aobHwzHoyG0n', 'type': 'tool_call'}], usage_metadata={'input_tokens': 54, 'output_tokens': 18, 'total_tokens': 72, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})


In [9]:
pprint(result_a.tool_calls)

[{'args': {'a': 2, 'b': 3},
  'id': 'call_XLI6Vla4MLH8aobHwzHoyG0n',
  'name': 'multiply',
  'type': 'tool_call'}]


In [15]:
multiply.invoke(result_a.tool_calls[0])

ToolMessage(content='6', name='multiply', tool_call_id='call_XLI6Vla4MLH8aobHwzHoyG0n')

In [16]:
llm_with_tools.invoke([result_a.tool_calls])

NotImplementedError: Unsupported message type: <class 'list'>
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/MESSAGE_COERCION_FAILURE 

In [41]:
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage

# Step 1: Define your tools
@tool
def add(a: int, b: int) -> int:
    """Adds two numbers."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiplies two numbers."""
    return a * b

tools = {
    "add": add,
    "multiply": multiply
}

tools_list = list(tools.values())

# Step 2: Initialize your language model
llm = model

# Step 3: Bind tools to the model
agent = llm.bind_tools(tools_list)

# Step 4: Define a function to process user queries
def process_query(user_query: str) -> str:
    # Invoke the agent with the user query
    response = agent.invoke(user_query)
    
    # Check if the response contains a tool call
    if response.tool_calls:
        tool_msgs = [HumanMessage(user_query),
                     response
                     ]
        for tool_call in response.tool_calls:
            # Extract tool name and arguments
            tool_name = tool_call["name"]

            # Invoke tools and append tool message
            tool_msgs.append(tools[tool_name].invoke(tool_call))
    
    
    # Invoke llm with Query and tool responses
    final_response = agent.invoke(tool_msgs)
    
    return final_response
    

In [40]:
# Example usage
user_query = "What is the result of adding 3 and 4?"
final_response = process_query(user_query)
print(final_response)

[HumanMessage(content='What is the result of adding 3 and 4?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5ACf41h2py4MCcQ7Rr0jTuzJ', 'function': {'arguments': '{"a":3,"b":4}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 79, 'total_tokens': 97, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-85a673e3-7952-44d8-8538-4735c7bc9005-0', tool_calls=[{'name': 'add', 'args': {'a': 3, 'b': 4}, 'id': 'call_5ACf41h2py4MCcQ7Rr0jTuzJ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 79, 'output_tokens': 18, 'total_tokens': 97, 'input_token_detail

In [26]:
response = agent.invoke(user_query)

In [33]:
tool_call = response.tool_calls[0]
tool_name = tool_call["name"]
tool_args = tool_call["args"]

tools[tool_name].invoke(tool_call)

ToolMessage(content='7', name='add', tool_call_id='call_RUVmH5OihmDkfBN7DDtuxkR8')

In [36]:
tool_call

{'name': 'add',
 'args': {'a': 3, 'b': 4},
 'id': 'call_RUVmH5OihmDkfBN7DDtuxkR8',
 'type': 'tool_call'}

In [37]:
response.tool_calls

[{'name': 'add',
  'args': {'a': 3, 'b': 4},
  'id': 'call_RUVmH5OihmDkfBN7DDtuxkR8',
  'type': 'tool_call'}]

In [38]:
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_RUVmH5OihmDkfBN7DDtuxkR8', 'function': {'arguments': '{"a":3,"b":4}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 79, 'total_tokens': 97, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d6b94590-96b2-4641-95a6-022e4c626e43-0', tool_calls=[{'name': 'add', 'args': {'a': 3, 'b': 4}, 'id': 'call_RUVmH5OihmDkfBN7DDtuxkR8', 'type': 'tool_call'}], usage_metadata={'input_tokens': 79, 'output_tokens': 18, 'total_tokens': 97, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [59]:
from langchain_core.tools import BaseTool
from langchain_core.messages import HumanMessage, ToolMessage
from typing import List

class ToolAgent:
    def __init__(self, model, tools: List[BaseTool]):
        """
        Initialize agent with LLM and list of tools
        
        Args:
            model: Language model instance
            tools: List of tools (must be @tool decorated or BaseTool instances)
        """
        self.llm = model
        self.tools = {tool.name: tool for tool in tools}
        self.agent = self.llm.bind_tools(tools)

    def invoke(self, user_query: str) -> str:
        """
        Process user query using available tools
        
        Args:
            user_query: Input question/request from user
            
        Returns:
            Final response after tool processing
        """
        # Initial agent response
        response = self.agent.invoke(user_query)
        
        # If no tool calls needed, return directly
        if not response.tool_calls:
            return response.content

        # Prepare message history with original query and agent response
        messages = [
            HumanMessage(content=user_query),
            response
        ]

        # Process each tool call
        for tool_call in response.tool_calls:
            tool_name = tool_call["name"]
            
            if tool_name not in self.tools:
                raise ValueError(f"Unknown tool: {tool_name}")

            # Get the tool instance
            tool = self.tools[tool_name]
            
            # Invoke the tool with the ENTIRE tool_call dictionary
            tool_result = tool.invoke(tool_call)
            
            # Create proper ToolMessage with call ID and result
            messages.append(tool_result)

        # Get final response with tool outputs
        self.messages = messages
        final_response = self.agent.invoke(messages)
        return final_response

In [60]:
tool_agent = ToolAgent(
    model=model,
    tools=tools_list
)

In [64]:
res = tool_agent.invoke("What would be 54 mulitplied by 26 plus 2")

In [67]:
pprint(tool_agent.messages)

[HumanMessage(content='What would be 54 mulitplied by 26 plus 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_utU03Kxl0dEbQqucQchLwp90', 'function': {'arguments': '{"a": 54, "b": 26}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_UDR4zhuI3ZjxKfsiHMBFotPk', 'function': {'arguments': '{"a": 54, "b": 2}', 'name': 'add'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 81, 'total_tokens': 132, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b681587a-5eae-481a-9614-802e1cd72789-0', tool_calls=[{'name': 'multiply', 'args': {'a': 54, 'b': 26}, 'id': 'call_